# import

In [2]:
from mlxtend.plotting import plot_decision_regions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Data

Data thu nhập với chỉ số liên quan đến chỉ số liên quan đến bệnh tiểu đường của bệnh nhân nữ và các chỉ số liên quan đến thận và tiêu hóa
- Pregnancies: số lần mang thai
- Glucose: Nồng độ đường glucose trong 2 lần đo
- BloodPressure: Huyết áp
- SkinThickness: Độ dày nết da
- Insulin: chỉ số Insulin(muU/ml)
- BMI: chỉ số cơ thể BMI
- DiabetesPedigreeFuction: giá trị đo chỉ số phản tiểu đường
- Age: Tuổi
- Label: 1 bị tiểu đường, 0 others

In [3]:
diabetes_data = pd.read_csv('D:\Bui Tuan\KNN\diabetes.csv')

In [5]:
diabetes_data.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [6]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Data Processing
- data có trường có value = 0 không có ý nghĩa như:
  - Glucose
  - BloodPressure
  - SkinThickness
  - Insulin
  - BMI

In [8]:
#value = 0
diabetes_data_copy = diabetes_data.copy(deep = True)
diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetes_data_copy[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [9]:
print(diabetes_data_copy.isnull().sum())

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


## scaling the data

In [13]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X =  pd.DataFrame(sc_X.fit_transform(diabetes_data_copy.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])

In [14]:
X.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.862287,-0.032746,0.558557,NaN,0.165097,0.468492,1.425995
1,-0.844885,-1.202229,-0.517645,-0.014657,NaN,-0.846404,-0.365061,-0.190672
2,1.233880,2.009241,-0.679278,NaN,NaN,-1.323254,0.604397,-0.105584
3,-0.844885,-1.071148,-0.517645,-0.587871,-0.518847,-0.629654,-0.920763,-1.041549
4,-1.141852,0.501816,-2.618874,0.558557,0.104968,1.537847,5.484909,-0.020496
5,0.342981,-0.186356,0.128887,NaN,NaN,-0.990904,-0.818079,-0.275760
6,-0.250952,-1.431620,-1.810709,0.271950,-0.569426,-0.210604,-0.676133,-0.616111
7,1.827813,-0.219126,NaN,NaN,NaN,0.410747,-1.020427,-0.360847
8,-0.547919,2.468022,-0.194379,1.513914,3.266189,-0.282854,-0.947944,1.681259
9,1.233880,0.108575,1.906851,NaN,NaN,NaN,-0.724455,1.766346


In [15]:
X.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,7.680000e+02,7.630000e+02,7.330000e+02,5.410000e+02,3.940000e+02,7.570000e+02,7.680000e+02,7.680000e+02
mean,2.544261e-17,1.316844e-16,-5.511735e-16,-2.842253e-17,4.790302e-18,3.805117e-16,2.398978e-16,1.857600e-16
std,1.000652e+00,1.000656e+00,1.000683e+00,1.000925e+00,1.001271e+00,1.000661e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-2.545803e+00,-3.911938e+00,-2.116442e+00,-1.193241e+00,-2.060204e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-7.434474e-01,-6.792777e-01,-6.834067e-01,-6.684780e-01,-7.163537e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.535857e-01,-3.274557e-02,-1.465704e-02,-2.575192e-01,-2.275351e-02,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.328967e-01,6.137865e-01,6.540926e-01,2.904259e-01,5.985967e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.533562e+00,4.008080e+00,6.672840e+00,5.820456e+00,5.005848e+00,5.883565e+00,4.063716e+00


In [32]:
y = diabetes_data_copy.Outcome

In [17]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

### chia dữ liệu

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/3,random_state=42, stratify=y)

In [36]:
from sklearn.neighbors import KNeighborsClassifier


test_scores = []
train_scores = []

for i in range(1, 15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [35]:
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

ValueError: max() arg is an empty sequence

In [31]:
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

ValueError: max() arg is an empty sequence